In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e8/sample_submission.csv
/kaggle/input/playground-series-s3e8/train.csv
/kaggle/input/playground-series-s3e8/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s3e8/train.csv")

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df = df.iloc[:,1:]

In [8]:
df.cut.unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [10]:
df.color.unique()

array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object)

In [23]:
df.clarity.unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
def ohe_cols(df, li):
    # create a new DataFrame with the two categorical columns
    cat_df = df[li]
    
    # create an instance of the OneHotEncoder and fit it to the categorical data
    ohe = OneHotEncoder()
    ohe.fit(cat_df)
    
    # transform the categorical data using the OneHotEncoder
    ohe_array = ohe.transform(cat_df).toarray()
    
    # create column names for the one-hot encoded columns
    col_names = ohe.get_feature_names(li)
    
    # create a new DataFrame with the one-hot encoded data and column names
    ohe_df = pd.DataFrame(ohe_array, columns=col_names)
    
    # concatenate the original DataFrame with the new one-hot encoded DataFrame
    df_ohe = pd.concat([df, ohe_df], axis=1)
    
    # drop the original categorical columns
    df_ohe.drop(columns=li, inplace=True)
    
    return df_ohe

In [24]:
train = ohe_cols(df,["cut","color","clarity"])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
x = train.drop("price",axis=1)
y = train.price

In [28]:
from sklearn.linear_model import LinearRegression

In [29]:
model1 = LinearRegression()

In [30]:
model1.fit(x,y)

LinearRegression()

In [32]:
test = pd.read_csv("/kaggle/input/playground-series-s3e8/test.csv")

In [34]:
test = ohe_cols(test, ["cut","color",'clarity'])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [35]:
test

,id,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,193573,0.35,62.3,56.0,4.51,4.54,2.82,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,193574,0.77,62.8,56.0,5.83,5.87,3.68,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,193575,0.71,61.9,53.0,5.77,5.74,3.55,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,193576,0.33,61.6,55.0,4.44,4.42,2.73,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,193577,1.20,62.7,56.0,6.75,6.79,4.24,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129045,322618,0.72,62.0,56.0,5.75,5.78,3.57,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
129046,322619,0.70,59.6,62.0,5.77,5.74,3.43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
129047,322620,1.01,62.3,58.0,6.44,6.41,4.01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
129048,322621,1.35,62.0,56.0,7.05,7.08,4.38,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
x2 = test.iloc[:,1:]

In [39]:
y_pred=model1.predict(x2)

In [42]:
sub = pd.DataFrame({"id":test.id,"price":y_pred})

In [44]:
sub.to_csv("submission1.csv",index=False)